In [1]:
!pip3 install pandas
!pip3 install xarray
!pip3 install netCDF4
!pip3 install gdal
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pylab
import xarray as xr
import netCDF4

from sklearn import tree, preprocessing
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split

  Using cached https://files.pythonhosted.org/packages/e5/57/7f0536cd46bebb30e709b8cd3bcebf9c3d4acc4ad5e9d7bfc73cd39c09a9/GDAL-2.3.2.tar.gz
    Complete output from command python setup.py egg_info:
    running egg_info
    creating pip-egg-info/GDAL.egg-info
    writing pip-egg-info/GDAL.egg-info/PKG-INFO
    writing dependency_links to pip-egg-info/GDAL.egg-info/dependency_links.txt
    writing top-level names to pip-egg-info/GDAL.egg-info/top_level.txt
    writing manifest file 'pip-egg-info/GDAL.egg-info/SOURCES.txt'
    Traceback (most recent call last):
      File "/tmp/pip-install-lk2f712p/gdal/setup.py", line 153, in fetch_config
        p = subprocess.Popen([command, args], stdout=subprocess.PIPE)
      File "/usr/lib/python3.6/subprocess.py", line 709, in __init__
        restore_signals, start_new_session)
      File "/usr/lib/python3.6/subprocess.py", line 1344, in _execute_child
        raise child_exception_type(errno_num, err_msg, err_filename)
    FileNotFoundError: [Er

In [2]:
cnx = sqlite3.connect('/home/vikram/Documents/code/wildfires/new_wildfire_data/fire_archive_V1_33596.csv')#2017
cnx1 = sqlite3.connect('/home/vikram/Documents/code/wildfires/new_wildfire_data/fire_archive_V1_33598.csv')#2018
cnx2 = sqlite3.connect('/home/vikram/Documents/code/wildfires/new_wildfire_data/fire_nrt_V1_33598.csv')#2018 recent


In [3]:
ds = xr.open_dataset('/home/vikram/Documents/code/wildfires/soil_moisture_data_1/ftp.bou.class.noaa.gov/3365939045/001/NPR_SMOPS_CMAP_D20170316.nc')
nc = netCDF4.Dataset('/home/vikram/Documents/code/wildfires/soil_moisture_data_1/ftp.bou.class.noaa.gov/3365939045/001/NPR_SMOPS_CMAP_D20170316.nc')

In [4]:
nc.variables['NRT_SMOS_SM'].add_offset

0.0

In [5]:
nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Satellite: ESA's SMOS, EUMETSAT's MetOp-A and MetOp-B, and JAXA's GCOM-W1
    Instrument: MIRAS on SMOS, ASCAT on MetOp-A, ASCAT on MetOp-B, and AMSR2 on GCOM-W1
    Projection: Plate Carree (Equirectangular Projection)
    Latitude_at_Lower_Left_Corner: -90.0
    Longitude_at_Lower_Left_Corner: -180.0
    Latitude_at_Upper_Right_Corner: 90.0
    Longitude_at_Upper_Right_Corner: 180.0
    Product_Resolution: 0.25 degree
    Date_Start: 20170316
    Date_End: 20170316
    Time_Start: 00:00 UTC
    Time_End: 23:59 UTC
    Byte_Order: Big endian
    Product_Version_Number: 2.0
    Data_Compression_Type: No Compression
    Production_Location: NOAA/NESDIS/OSPO at Camp Springs, Maryland, USA
    Contact_Information: Science Lead  : Xiwu Zhan,  NOAA/NESDIS/STAR, xiwu.zhan@noaa.gov
      Operation Lead: Limin Zhao, NOAA/NESDIS/OSPO, limin.zhao@noaa.gov
    dimensions(sizes): Longitude(1440), L

In [6]:

soil_df = ds.to_dataframe()
soil_df.head()

Blended_SM  NRT_SMOS_SM  SMOS_SM  ASCAT_A_SM  ASCAT_B_SM  \
Latitude Longitude                                                             
0        0             -0.0999      -0.0999  -0.0999     -0.0999     -0.0999   
         1             -0.0999      -0.0999  -0.0999     -0.0999     -0.0999   
         2             -0.0999      -0.0999  -0.0999     -0.0999     -0.0999   
         3             -0.0999      -0.0999  -0.0999     -0.0999     -0.0999   
         4             -0.0999      -0.0999  -0.0999     -0.0999     -0.0999   

                    AMSR2_SM  Spare_SM1  Blended_hour  Blended_minute  \
Latitude Longitude                                                      
0        0           -0.0999       -999           -99             -99   
         1           -0.0999       -999           -99             -99   
         2           -0.0999       -999           -99             -99   
         3           -0.0999       -999           -99             -99   
         4           -0.0999       -999           -99             -99   

                    NRT_SMOS_hour    ...      AMSR2_minute  Spare1_hour  \
Latitude Longitude                   ...                                  
0        0                    -99    ...               -99          -99   
         1                    -99    ...               -99          -99   
         2                    -99    ...               -99          -99   
         3                    -99    ...               -99          -99   
         4                    -99    ...               -99          -99   

                    Spare1_minute  Blended_QA  NSMOS_QA  SMOS_QA  ASCAT_A_QA  \
Latitude Longitude                                                             
0        0                    -99        -999      -999     -999        -999   
         1                    -99        -999      -999     -999        -999   
         2                    -99        -999      -999     -999        -999   
         3                    -99        -999      -999     -999        -999   
         4                    -99        -999      -999     -999        -999   

                    ASCAT_B_QA  AMSR2_QA  Spare1_QA  
Latitude Longitude                                   
0        0                -999      -999       -999  
         1                -999      -999       -999  
         2                -999      -999       -999  
         3                -999      -999       -999  
         4                -999      -999       -999  

[5 rows x 28 columns]

In [7]:
c_soildf = soil_df[soil_df['Blended_SM'] != -0.0999]

In [15]:
c_soildf.tail()

Blended_SM  NRT_SMOS_SM  SMOS_SM  ASCAT_A_SM  ASCAT_B_SM  \
Latitude Longitude                                                             
577      574            0.3964      -0.0999  -0.0999       0.311     -0.0999   
578      451            0.2006      -0.0999  -0.0999       0.584      0.4900   
         452            0.1815      -0.0999  -0.0999       0.600      0.6630   
         574            0.3640      -0.0999  -0.0999       0.392     -0.0999   
         575            0.4346      -0.0999  -0.0999       0.397     -0.0999   

                    AMSR2_SM  Spare_SM1  Blended_hour  Blended_minute  \
Latitude Longitude                                                      
577      574         -0.0999       -999            11              38   
578      451          0.0381       -999             5              13   
         452          0.0389       -999            18              43   
         574         -0.0999       -999            23              48   
         575         -0.0999       -999            23              48   

                    NRT_SMOS_hour ...   Spare1_minute  Blended_QA  NSMOS_QA  \
Latitude Longitude                ...                                         
577      574                  -99 ...             -99        8197      -999   
578      451                  -99 ...             -99        3775      -999   
         452                  -99 ...             -99       21915      -999   
         574                  -99 ...             -99        6661      -999   
         575                  -99 ...             -99        6917      -999   

                    SMOS_QA  ASCAT_A_QA  ASCAT_B_QA  AMSR2_QA  Spare1_QA    x  \
Latitude Longitude                                                              
577      574           -999        6150        -999      -999       -999  574   
578      451           -999        1548        1545      3775       -999  451   
         452           -999         778         778     21915       -999  452   
         574           -999        6918        -999      -999       -999  574   
         575           -999        6917        -999      -999       -999  575   

                      y  
Latitude Longitude       
577      574        577  
578      451        578  
         452        578  
         574        578  
         575        578  

[5 rows x 30 columns]

In [14]:
c_soildf['x'] = c_soildf.index.get_level_values(1)
c_soildf['y'] = c_soildf.index.get_level_values(0)

/home/vikram/.virtualenvs/pw/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/vikram/.virtualenvs/pw/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
def convLat(row):
    y = (((row.name[0] * 1.0) + 90.0) * (720 / 180.0))
    return y

In [ ]:
def convLong(row):
    x = ((row.name[1] + 180.0) * (1440 / 360.0))
    return x

In [ ]:
def getValLat(row):
    return row.name[0]

In [ ]:
def getValLong(row):
    return row.name[1]

In [ ]:
df = pd.read_csv("SELECT latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight FROM 'Fires'", cnx)
df['DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
print(df.head())

In [ ]:
df['MONTH'] = pd.DatetimeIndex(df['DATE']).month
df['DAY_OF_WEEK'] = df['DATE'].dt.weekday_name
df_orig = df.copy()
print(df.head())

In [ ]:
df['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='coral')
#plt.show()
pylab.savefig('causes.png')

In [ ]:
df['STATE'].value_counts().head(n=10).plot(kind='barh',color='coral')
pylab.savefig('state.png')

In [ ]:
#le = preprocessing.LabelEncoder()
#df['STAT_CAUSE_DESCR'] = le.fit_transform(df['STAT_CAUSE_DESCR'])
#df['STATE'] = le.fit_transform(df['STATE'])
#df['DAY_OF_WEEK'] = le.fit_transform(df['DAY_OF_WEEK'])

In [ ]:
def plot_corr(df,size=10):
    corr = df.corr()  #the default method is pearson
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr,cmap=plt.cm.Oranges)
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)    
    pylab.savefig('matrix.png')
plot_corr(df)


In [ ]:
df.isnull().any() #Checks for null values

In [ ]:
# Find number of wildfires from 03-12-2016 to 03-12-2018
df['FIRE_YEAR'] = df['FIRE_YEAR'].astype(int)


In [ ]:
recent_df = df[df['FIRE_YEAR'] > 2003]
len(df[df['FIRE_YEAR'] > 2016])

In [ ]:
len(df[(df['FIRE_YEAR'] > 2014)])

In [ ]:
recent_df['STATE'].value_counts().head(n=10).plot(kind='barh',color='coral')

In [ ]:
recent_df.head()

In [ ]:
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

In [ ]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

In [ ]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

In [ ]:
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

In [ ]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Select the best split point for a dataset
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

# Print a decision tree
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
tree = build_tree(df, 1, 1)
print_tree(tree)